# Lecture 8
# Deep Learning for Computer Vision

## Computer vision
Most successful applications were initially CV applications
* ImageNet classification (2011 – )
    * 1000 classes
    * 1 million examples

Other Computer Vision applications
* OCR, image search, autonomous driving, medical diagnosis, etc

Convolutional network (convnet) is often used in CV tasks
* Cf. densely-connected (DCN) or fully-connected network (FCN)

**First example of convnet**

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(28, 28, 1)) ## Different from densenet input
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)

outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary() ## Check the output of the last conv layer

2022-03-10 11:59:43.612630: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-10 11:59:43.612991: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (N

In [2]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)) ## Check the shape
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1)) ## Check the shape
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.fit(train_images, train_labels, epochs=5, batch_size=64)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

2022-03-10 12:01:59.486139: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/5


2022-03-10 12:01:59.756078: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 12s 11ms/step - loss: 0.1534 - accuracy: 0.9523
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 0.0442 - accuracy: 0.9862
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 0.0306 - accuracy: 0.9902
Epoch 4/5
938/938 [==============================] - 10s 11ms/step - loss: 0.0222 - accuracy: 0.9931
Epoch 5/5
 42/313 [===>..........................] - ETA: 1s - loss: 0.0440 - accuracy: 0.9829

2022-03-10 12:02:51.393047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 5ms/step - loss: 0.0287 - accuracy: 0.9914
Test accuracy: 0.991


Dense layers learn global patterns, conv layers learn local pattern within the window (e.g., 3 x 3 kernel)
* Assumption: an image can be broken into local patterns such as edges, textures, etc

Key characteristics of convnets:

* Translation-invariant: an object/pattern can appear in different location of the image
* Can learn spatial hierarchies of patterns
* Efficient learning of increasing increasingly complex and abstract concepts

Convolution operates over rank-3 tensors, called feature maps
* Feature map: height x width x depth (or channel)
* Feature map for the input layer: for MNIST, 28 x 28 x 1 and for CIFAR10, 32 x 32 x 3
    * depth is 1 for grayscale images and 3 for RGB images
* Output is also a rank-3 feature map of different shape
    * Shape can change (26 x 26 x 32)
    * Depth is the number of filters/kernels in the conv layer